In [10]:
import numpy as np
import matplotlib.pyplot as plt
import inspect
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import os

In [3]:
import importlib
import sys
sys.path.append('../')
from Utils.perturbationsnew import RBPIndividual, RBPIndividualNew1, RBPIndividualNew1fast, RBPIndividualNew2, zeroPerturb, noisePerturb, blurPerturb,multisamplebackgroundIdentification
from Utils.models import *
from Utils.data import perturb, generateSynthetic, generateSyntheticPerturbReal,ReadTS

print(sys.path)


['/Users/gali/anaconda3/envs/LIMEtf/lib/python310.zip', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/lib-dynload', '', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages/setuptools/_vendor', '../', '../']


## Test important content removal (accuracy decrease)

### Load synthetic dataset and train CNN model 

In [4]:
x_train, y_train,x_test_original, y_test_original = ReadTS('Strawberry')
y_train = y_train - 1
y_test_original = y_test_original - 1
print(x_train.shape)
#print(y_train)
#print(x_train[1])

(613, 235, 1)


In [23]:
from sklearn.preprocessing import StandardScaler
def normalize(TS):

    original_shape = TS.shape
    
    TS_2d = TS.reshape(original_shape[0] , original_shape[1])
    
    scaler = StandardScaler()
    
    TS_normalized = scaler.fit_transform(TS_2d)
    
    TS = TS_normalized.reshape(original_shape)
    
    return TS

x_train = normalize(x_train)
x_test_original = normalize(x_test_original)
#print(x_train[1])

In [33]:
model = make_CNN_model(x_train.shape[1:],num_classes = 2)
trained_model, history = train_CNN_model(model, x_train, y_train, epochs=100, batch_size=32)

Epoch 1/100
16/16 [==============================] - 1s 24ms/step - loss: 0.4977 - sparse_categorical_accuracy: 0.7347 - val_loss: 0.6600 - val_sparse_categorical_accuracy: 0.5528
Epoch 2/100
16/16 [==============================] - 0s 19ms/step - loss: 0.4746 - sparse_categorical_accuracy: 0.7653 - val_loss: 0.6658 - val_sparse_categorical_accuracy: 0.7642
Epoch 3/100
16/16 [==============================] - 0s 18ms/step - loss: 0.4738 - sparse_categorical_accuracy: 0.7245 - val_loss: 0.6621 - val_sparse_categorical_accuracy: 0.6016
Epoch 4/100
16/16 [==============================] - 0s 18ms/step - loss: 0.4524 - sparse_categorical_accuracy: 0.7735 - val_loss: 0.6766 - val_sparse_categorical_accuracy: 0.5285
Epoch 5/100
16/16 [==============================] - 0s 18ms/step - loss: 0.4515 - sparse_categorical_accuracy: 0.7633 - val_loss: 0.7003 - val_sparse_categorical_accuracy: 0.4715
Epoch 6/100
16/16 [==============================] - 0s 18ms/step - loss: 0.4660 - sparse_categorica

In [34]:
test_loss, test_acc = trained_model.evaluate(x_test_original, y_test_original)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

12/12 [==============================] - 0s 4ms/step - loss: 0.5317 - sparse_categorical_accuracy: 0.7541
Test accuracy 0.754054069519043
Test loss 0.5317051410675049


In [35]:
def l2norm(perturb):
    difference = perturb - x_test_original
    
    squared_difference = np.square(difference)
    
    sum_of_squares = np.sum(squared_difference, axis=1)
    
    l2_norms = np.sqrt(sum_of_squares)
    
    average_l2_norm = np.mean(l2_norms)
    
    print(average_l2_norm)

    return average_l2_norm


def perturb_predict(strategy,index_0,index_1):
    if strategy =='RBP1fast':
        bg = multisamplebackgroundIdentification(perturb_test_set)
        x_test_pert = np.asarray([perturb(strategy,x,index_0,index_1,bg = bg) for x in perturb_test_set])
        x_test_pert = np.expand_dims(x_test_pert, axis=-1)
        predictions_pert = model.predict(x_test_pert)
        predictions_pert = np.argmax(predictions_pert,axis=1)
    else:
        x_test_pert = np.asarray([perturb(strategy,x,index_0,index_1) for x in perturb_test_set])
        x_test_pert = np.expand_dims(x_test_pert, axis=-1)
        predictions_pert = model.predict(x_test_pert)
        predictions_pert = np.argmax(predictions_pert,axis=1)

    accuracy = accuracy_score(y_test_original, predictions_pert)
    precision = precision_score(y_test_original, predictions_pert, average='weighted')  
    recall = recall_score(y_test_original, predictions_pert, average='weighted')
    f1 = f1_score(y_test_original, predictions_pert, average='weighted')
    l2 = l2norm(x_test_pert)

    # print(strategy)
    # print(accuracy)
    # print(precision)
    # print(recall)
    # print(f1)

    return accuracy,precision,recall,f1,l2

In [37]:
perturb_test_set = x_test_original.copy()
perturb_test_set = perturb_test_set.reshape(x_test_original.shape[0:2])
index_0 = 0
index_1 = 80

strategies = ['RBP', 'RBP1fast', 'zero', 'noise', 'blur']

results = []
l2_results = []

for strategy in strategies:
    accuracy, precision, recall, f1, l2 = perturb_predict(strategy, index_0, index_1)
    results.append([strategy, accuracy, precision, recall, f1])
    l2_results.append(l2)

columns = ['Strategy', 'Accuracy', 'Precision', 'Recall', 'F1 Score']
df_results = pd.DataFrame(results, columns=columns)
l2_columns = ['L2-Norm']
l2_results = pd.DataFrame(l2_results, columns=l2_columns)

predictions_original = model.predict(x_test_original)
predictions_original = np.argmax(predictions_original,axis=1)
accuracy = accuracy_score(y_test_original, predictions_original)
precision = precision_score(y_test_original, predictions_original, average='weighted')  
recall = recall_score(y_test_original, predictions_original, average='weighted')
f1 = f1_score(y_test_original, predictions_original, average='weighted')
original_data = ['original', accuracy, precision, recall, f1]
df_results.loc[-1] = original_data  
df_results.index = df_results.index + 1  
df_results = df_results.sort_index()  

df_results.loc[2, 'Strategy'] = 'MRBP'

current_directory = os.getcwd()
result_folder = os.path.join(current_directory, 'result_data')

csv_path = os.path.join(result_folder, 'Strawberry.csv')
df_results.to_csv(csv_path, index=False)

csv_path = os.path.join(result_folder, 'Strawberryl2.csv')
l2_results.to_csv(csv_path, index=False)

print("done")
print(df_results)
print(l2_results)

12/12 [==============================] - 0s 4ms/step
5.553233161058392
12/12 [==============================] - 0s 4ms/step
7.9522756962756365
12/12 [==============================] - 0s 4ms/step
7.542306225327786
12/12 [==============================] - 0s 5ms/step
12.112132189530518
12/12 [==============================] - 0s 4ms/step
0.5510306006617007
12/12 [==============================] - 0s 4ms/step
done
   Strategy  Accuracy  Precision    Recall  F1 Score
0  original  0.754054   0.791610  0.754054  0.717506
1       RBP  0.716216   0.734905  0.716216  0.670336
2      MRBP  0.656757   0.776187  0.656757  0.533808
3      zero  0.724324   0.740459  0.724324  0.684228
4     noise  0.643243   0.413762  0.643243  0.503592
5      blur  0.754054   0.791610  0.754054  0.717506
     L2-Norm
0   5.553233
1   7.952276
2   7.542306
3  12.112132
4   0.551031


/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
